In [26]:
#source: https://community.intersystems.com/post/save-pandas-dataframe-iris-quick-note
#https://github.com/intersystems-community/iris-driver-distribution/tree/main/JDBC/JDK18

import jaydebeapi
url = "jdbc:IRIS://localhost:1972/USER" 
driver = 'com.intersystems.jdbc.IRISDriver'
user = "_SYSTEM"
password = "SYS"
jarfile = "/opt/irisbuild/intersystems-jdbc-3.6.1.jar"

In [6]:
!pip install jaydebeapi

Defaulting to user installation because normal site-packages is not writeable


In [9]:
!pwd

/opt/irisbuild


In [27]:
conn = jaydebeapi.connect(driver, url, [user, password], jarfile)
curs = conn.cursor()

In [55]:
def to_sql_iris(cursor, dataFrame, tableName, schemaName='SQLUser', drop_table=False ):
        """"
        Dynamically insert dataframe into an IRIS table via SQL by "excutemany" 
        
        Inputs:
            cursor:      Python JDBC or PyODBC cursor from a valid and establised DB connection
            dataFrame:   Pandas dataframe
            tablename:   IRIS SQL table to be created, inserted or apended
            schemaName:  IRIS schemaName, default to "SQLUser"
            drop_table:  If the table already exsits, drop it and re-create it if True; othrewise keep it and appen 
        Output:
            True is successful; False if there is any exception.
        """
        if drop_table:   
            try:                 
                curs.execute("DROP TABLE %s.%s" %(schemaName, tableName))
            except Exception:
                pass
        
        try: 
            dataFrame.columns = dataFrame.columns.str.replace("[() -]", "_")
            curs.execute(pd.io.sql.get_schema(dataFrame, tableName))
        except Exception:
            pass
        
        curs.fast_executemany = True
        cols = ", ".join([str(i) for i in dataFrame.columns.tolist()])
        wildc =''.join('?, ' * len(dataFrame.columns))
        wildc = '(' + wildc[:-2] + ')'
        sql = "INSERT INTO " + tableName + " ( " + cols.replace('-', '_') + " ) VALUES" + wildc
        #print(sql)
        curs.executemany(sql, list(dataFrame.itertuples(index=False, name=None)) )
        return True

In [56]:
#criar DF aqui
import pandas as pd
fiocruz=pd.read_csv('/home/irisowner/TRAJETORIAS_DATASET_Population_indicators.csv', sep=',')




In [66]:
pd.io.sql.get_schema(fiocruz)

TypeError: get_schema() missing 1 required positional argument: 'name'

In [41]:
fiocruz.info(show_counts=True, memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 772 entries, 0 to 771
Data columns (total 1 columns):
 #   Column                                                                                                                                                                                                                                                     Non-Null Count  Dtype 
---  ------                                                                                                                                                                                                                                                     --------------  ----- 
 0   state_abbrev,"state","municipality","geocode","urb2000","rur2000","tot2000","prop_urb2000","prop_rur2000","urb2010","rur2010","tot2010","prop_urb2010","prop_rur2010","pop_estimated2006","pop_estimated2017","urb2006e","rur2006e","urb2017e","rur2017e"  772 non-null    object
dtypes: object(1)
memory usage: 6.2+ KB


In [50]:
#listar colunas
import io
buffer = io.StringIO()
fiocruz.info(buf=buffer)
lines = buffer.getvalue().splitlines()
ndf = (pd.DataFrame([x.split() for x in lines[5:-2]], columns=lines[3].split())
       .drop('Count',axis=1)
       .rename(columns={'Non-Null':'Non-Null Count'}))
print (ndf)

   #                                             Column Non-Null Count   Dtype
0  0  state_abbrev,"state","municipality","geocode",...            772  object


In [38]:
lines

["<class 'pandas.core.frame.DataFrame'>",
 'RangeIndex: 772 entries, 0 to 771',
 'Data columns (total 1 columns):',
 ' #   Column                                                                                                                                                                                                                                                     Non-Null Count  Dtype ',
 '---  ------                                                                                                                                                                                                                                                     --------------  ----- ',
 ' 0   state_abbrev,"state","municipality","geocode","urb2000","rur2000","tot2000","prop_urb2000","prop_rur2000","urb2010","rur2010","tot2010","prop_urb2010","prop_rur2010","pop_estimated2006","pop_estimated2017","urb2006e","rur2006e","urb2017e","rur2017e"  772 non-null    object',
 'dtypes: object(1)',
 'memory usag

In [62]:
iris_schema = 'SQLUser'
iris_table = 'populacao'

In [63]:
#df2iris
to_sql_iris(curs, fiocruz, iris_table, drop_table=True)


/tmp/ipykernel_889/2879127366.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  dataFrame.columns = dataFrame.columns.str.replace("[() -]", "_")


java.sql.SQLException: java.sql.SQLException: [SQLCODE: <-30>:<Table or view not found>]
[Location: <Prepare>]
[%msg: < Table 'SQLUSER.POPULACAO' not found>]

In [65]:
to_sql_iris(curs, fiocruz, iris_table, iris_schema, drop_table=True)   # save it into a new IRIS table of specified name
#to_sql_iris(curs, df, iris_table)  # append dataframe to an exsiting IRIS table

/tmp/ipykernel_889/2879127366.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  dataFrame.columns = dataFrame.columns.str.replace("[() -]", "_")


java.sql.SQLException: java.sql.SQLException: [SQLCODE: <-30>:<Table or view not found>]
[Location: <Prepare>]
[%msg: < Table 'SQLUSER.POPULACAO' not found>]

In [ ]:
import pandas as pd
from IPython.display import display

In [ ]:
#iris2df
df2 = pd.read_sql("SELECT COUNT(*) from %s.%s" %(iris_schema, iris_table),conn)
display(df2)